## 实验题目5 高斯(Gauss)列主元消去法

### 代码实现

In [33]:
using Printf
using LinearAlgebra

In [34]:
# from: https://stackoverflow.com/questions/58667332/is-there-a-way-to-swap-columns-in-o1-in-julia
function swapcols!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 1)
        X[k, i], X[k, j] = X[k, j], X[k, i]
    end
end
# from: https://discourse.julialang.org/t/swap-cols-rows-of-a-matrix/47904/9
function _swapcol!(x, i, j)
    for k in axes(x, 1)  # <- give dimension as input to axes function
        x[k, i], x[k, j] = x[k, j], x[k, i]
    end
end

_swapcol! (generic function with 1 method)

In [35]:
function swaprows!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 2)
        X[i, k], X[j, k] = X[j, k], X[i, k]
    end
end

swaprows! (generic function with 1 method)

In [36]:
# https://stackoverflow.com/questions/45396685/what-does-an-exclamation-mark-mean-after-the-name-of-a-function
# https://people.richland.edu/james/lecture/m116/matrices/pivot.html
function pivoting!(A::Matrix{Float64}, k::Integer, n::Integer)
    val, idx = findmax(A[k:n, k])
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end
function pivoting!(A::Matrix{Float64}, b::Vector{Float64}, k::Integer, n::Integer, implicit::Bool)
    s = [maximum(A[i, k:n]) for i in k:n]
    if 0 in s
        println("Cannot solve a singular matrix!")
        return
    end
    if implicit
        val, idx = findmax(A[k:n, k] ./ s[1:n-k+1])
    else
        A[k:n, k:n] = A[k:n, k:n] ./ s
        b[k:n] = b[k:n] ./ s
        val, idx = findmax(A[k:n, k])
    end
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end

pivoting! (generic function with 2 methods)

In [37]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, k, n)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

In [38]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64}, implicit::Bool)
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, b, k, n, implicit)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

### 测试代码

#### Test 1 - Correctness

In [46]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)
    try
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b), 2))\t")

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, true), 2))\t")

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, false), 2))\t")
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

8.425059100878904e-13	  0.165326 seconds (180.31 k allocations: 439.810 MiB, 17.78% gc time)
3.4350627540995605e-12	  0.208177 seconds (227.26 k allocations: 515.581 MiB, 15.69% gc time)
1.507283366551913e-12	  0.242366 seconds (228.37 k allocations: 653.650 MiB, 14.92% gc time)

2.6375399335317267e-12	  0.168019 seconds (180.32 k allocations: 439.810 MiB, 18.44% gc time)
7.094038267623084e-13	  0.220397 seconds (227.26 k allocations: 515.581 MiB, 16.62% gc time)
6.9047956070794795e-12	  0.251966 seconds (228.37 k allocations: 653.650 MiB, 15.47% gc time)

5.941176607300328e-13	  0.181817 seconds (180.31 k allocations: 439.810 MiB, 17.66% gc time)
3.5664035006884525e-13	  0.244468 seconds (227.26 k allocations: 515.581 MiB, 14.90% gc time)
9.296910460489593e-13	  0.246992 seconds (228.37 k allocations: 653.650 MiB, 14.85% gc time)

1.5634026966848953e-11	  0.160013 seconds (180.32 k allocations: 439.810 MiB, 18.44% gc time)
2.3075886211457332e-12	  0.224174 seconds (227.26 k allocation

#### Test 2 - Performance 

In [43]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)
    try
        @time A \ b

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b)

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, true)

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, false)
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

  0.006502 seconds (4 allocations: 708.172 KiB)
  0.182775 seconds (180.30 k allocations: 439.115 MiB, 21.88% gc time)
  0.239597 seconds (227.24 k allocations: 514.886 MiB, 18.46% gc time)
  0.275982 seconds (228.35 k allocations: 652.955 MiB, 16.01% gc time)

  0.006066 seconds (4 allocations: 708.172 KiB)
  0.182836 seconds (180.30 k allocations: 439.115 MiB, 20.52% gc time)
  0.229287 seconds (227.24 k allocations: 514.886 MiB, 18.00% gc time)
  0.267037 seconds (228.35 k allocations: 652.955 MiB, 16.77% gc time)

  0.007162 seconds (4 allocations: 708.172 KiB)
  0.285603 seconds (180.30 k allocations: 439.115 MiB, 43.31% gc time)
  0.211279 seconds (227.24 k allocations: 514.886 MiB, 16.37% gc time)
  0.241856 seconds (228.35 k allocations: 652.955 MiB, 15.15% gc time)

  0.007173 seconds (4 allocations: 708.172 KiB)
  0.170964 seconds (180.30 k allocations: 439.115 MiB, 19.40% gc time)
  0.215837 seconds (227.24 k allocations: 514.886 MiB, 17.98% gc time)
  0.257109 seconds (228.

#### Test 3 - Special Matrix

Bug: 处理上三角矩阵计算有异常

Todo: 应当增加对于特殊矩阵的测试，待测试列表同Julia Special Matrix:

- [ ] Symmetric
- [ ] Hermitian
- [ ] UpperTriangular
- [ ] UnitUpperTriangular
- [ ] LowerTriangular
- [ ] UnitLowerTriangular
- [ ] UpperHessenberg
- [ ] Tridiagonal
- [ ] SymTridiagonal
- [ ] Bidiagonal
- [ ] Diagonal
- [ ] UniformScaling


### 实验题目

#### 问题 1

In [66]:
A = [0.4096 0.1234 0.3678 0.2943
    0.2246 0.3872 0.4015 0.1129
    0.3645 0.1920 0.3781 0.0643
    0.1784 0.4002 0.2786 0.3927]
b = [1.1951; 1.1262; 0.9989; 1.2499]
# @time A \ b
# @time display(gauss(4, A, b))
display(@time gauss(4, A, b,true))
display(@time gauss(4, A, b))

4-element Vector{Float64}:
 1.0000000000000169
 1.0000000000000142
 0.9999999999999768
 0.9999999999999994

4-element Vector{Float64}:
 -1.3477154991442357
 -1.0691119494363102
  4.308734864083523
  0.9999999999999994

  0.000023 seconds (61 allocations: 4.781 KiB)
  0.000013 seconds (34 allocations: 3.031 KiB)
